In [1]:
import os

In [2]:
import numpy as np
import pandas as pd
import healpy as hp
import GCRCatalogs

In [3]:
from sqlalchemy import create_engine

In [4]:
cat = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_image')
cosmodc2_nside = 32

In [5]:
path = '/global/projecta/projectdirs/lsst/groups/SSim/DC2/cosmoDC2_v1.1.4/sne_cosmoDC2_v1.1.4_MS_DDF.db'

In [6]:
os.path.join('sqlite:///', path)

'/global/projecta/projectdirs/lsst/groups/SSim/DC2/cosmoDC2_v1.1.4/sne_cosmoDC2_v1.1.4_MS_DDF.db'

In [7]:
engine = create_engine('sqlite:///' + path)

In [8]:
sndf = pd.read_sql_table('sne_params', con=engine)

In [9]:
sndf.head()

,htmid_level_6,galaxy_id,c_in,mB,t0_in,x0_in,x1_in,z_in,snid_in,snra_in,sndec_in
0,9021,10562502111,0.035201,18.516489,60772.368515,0.000617,1.477374,0.077278,MS_10199_0,66.115587,-40.866055
1,9023,10562500822,-0.086665,17.966773,63060.297448,0.001028,0.576378,0.073256,MS_10199_1,66.662435,-42.042877
2,9021,10562500735,0.037553,19.078334,62832.166508,0.000370,-0.177457,0.068469,MS_10199_2,65.212146,-41.416473
3,9021,10562502246,0.076715,19.192019,61400.008116,0.000335,-1.344205,0.080189,MS_10199_3,65.469824,-41.109646
4,9023,10562504918,-0.012653,19.388848,61342.657248,0.000278,0.474204,0.115499,MS_10199_5,65.986211,-42.086708


In [12]:
list(x for x in cat.list_all_native_quantities() if 'morphology' in x)

['morphology/positionAngle',
 'morphology/totalAxisRatio',
 'morphology/diskMinorAxisArcsec',
 'morphology/spheroidEllipticity2',
 'morphology/spheroidHalfLightRadiusArcsec',
 'morphology/spheroidMinorAxisArcsec',
 'morphology/totalEllipticity',
 'morphology/diskEllipticity1',
 'morphology/diskHalfLightRadiusArcsec',
 'morphology/diskHalfLightRadius',
 'morphology/spheroidEllipticity',
 'morphology/diskEllipticity',
 'morphology/spheroidSersicIndex',
 'morphology/totalEllipticity1',
 'morphology/totalSersicIndex',
 'morphology/totalEllipticity2',
 'morphology/diskMajorAxisArcsec',
 'morphology/spheroidAxisRatio',
 'morphology/diskEllipticity2',
 'morphology/spheroidHalfLightRadius',
 'morphology/diskSersicIndex',
 'morphology/diskAxisRatio',
 'morphology/spheroidEllipticity1',
 'morphology/spheroidMajorAxisArcsec']

In [10]:
sndf = sndf.query('snra_in < 53.77 and snra_in > 52.479 and sndec_in < - 27.533 and sndec_in > -28.667')

In [11]:
your_data = sndf.query('snra_in < 53.77 and snra_in > 52.479 and sndec_in < - 27.533 and sndec_in > -28.667')[['galaxy_id',  'snra_in', 'sndec_in', 'snid_in']].values

In [25]:
len(your_data)

4607

In [15]:
# change these quantities as needed
#your_data = [(9687500000, 62.58, -40.33), (10500000002, 62.32, -41.86)]
quantities_needed = ['galaxy_id', 'ra', 'dec', 'mag_r', 'morphology/totalSersicIndex', 'morphology/spheroidHalfLightRadius', 'morphology/diskHalfLightRadius']

In [16]:
host_galaxy_props = []
hostless_props = []
for host_id, sn_ra, sn_dec, snid in your_data:
    pixels = [hp.ang2pix(cosmodc2_nside, sn_ra, sn_dec, lonlat=True)]
    pixels.extend(hp.get_all_neighbours(cosmodc2_nside, sn_ra, sn_dec, lonlat=True))
    for pixel in pixels:
        # remove 'redshift_block_lower == 0' in `native_filters` below if expecting high-z hosts
        data = cat.get_quantities(
            quantities_needed, 
            filters='galaxy_id == {}'.format(host_id), 
            native_filters=['healpix_pixel == {}'.format(pixel), 'redshift_block_lower == 0']
        )
        if len(data[quantities_needed[0]]):
            data['snid'] = snid
            host_galaxy_props.append(pd.DataFrame(data))
            break
    else:
        hostless_props.append([host_id, snid])
        # print('Cannot find host {} near ({}, {})'.format(host_id, sn_ra, sn_dec))

host_galaxy_props = pd.concat(host_galaxy_props, ignore_index=True)

NameError: name 'hbost_galaxy_props' is not defined

In [19]:
host_galaxy_props = pd.concat(host_galaxy_props, ignore_index=True)

In [20]:
len(host_galaxy_props)

2783

In [29]:
res = host_galaxy_props.set_index('snid').join(sndf.rename(columns=dict(snid_in='snid')).set_index('snid'), rsuffix='_sn')

In [30]:
res.head()

,morphology/diskHalfLightRadius,galaxy_id,morphology/spheroidHalfLightRadius,ra,morphology/totalSersicIndex,mag_r,dec,htmid_level_6,galaxy_id_sn,c_in,mB,t0_in,x0_in,x1_in,z_in,snra_in,sndec_in
snid,,,,,,,,,,,,,,,,,
MS_9043_2,1.944731,2562500806,0.985439,53.491017,3.850073,18.892840,-27.698565,9198,2562500806,-0.025506,18.022261,61571.389041,0.000973,1.499360,0.072476,53.490547,-27.698978
MS_9043_3,3.288042,2562500869,1.351803,53.228259,1.000000,20.105118,-28.654434,9198,2562500869,-0.007524,19.038116,61213.960335,0.000387,-1.932523,0.060312,53.226311,-28.654498
MS_9043_7,3.778045,2562504877,1.997927,52.707182,3.876497,18.045374,-28.554180,9198,2562504877,-0.078385,18.766703,63097.750601,0.000491,0.818096,0.104969,52.705649,-28.554953
MS_9043_8,3.230932,2562504524,1.964645,52.889861,2.191405,18.638982,-28.202155,9198,2562504524,-0.007228,19.459478,61929.959443,0.000259,1.108244,0.113109,52.889331,-28.202310
MS_9043_9,5.619791,2562504813,2.776914,53.162164,1.616776,18.168871,-27.799101,9198,2562504813,-0.012755,19.591836,59966.699846,0.000231,-0.092340,0.114249,53.162353,-27.799423


In [48]:
res.to_csv('host_sn_pairs.csv')

In [34]:
%matploitlib inline

UsageError: Line magic function `%matploitlib` not found.


In [51]:
len(res.query('z_in < 0.5'))/ 10 / 12

4.083333333333333

In [40]:
len(hostless_props) /10.0

182.4

In [41]:
1/ 4. / 12. 

0.020833333333333332

In [42]:
len(hostless_props)

1824

In [44]:
not_found = pd.DataFrame(hostless_props, columns=['host_id', 'sn_id'])

In [45]:
not_found.head()

,host_id,sn_id
0,1940857362,mDDF_8915_6971
1,1941191110,mDDF_8915_7651
2,1,hl_ms_1
3,390,hl_ms_390
4,1309,hl_ms_1309


In [46]:
not_found.to_csv('host_not.csv', index=False)

In [47]:
!pwd

/global/u2/r/rbiswas


In [53]:
len(xx)

2783

In [31]:
len(host_galaxy_props)

2783

In [52]:
xx =pd.merge(sndf, host_galaxy_props, on='snid_in').dropna()

In [34]:
sndf.columns

Index(['htmid_level_6', 'galaxy_id', 'c_in', 'mB', 't0_in', 'x0_in', 'x1_in',
       'z_in', 'snid_in', 'snra_in', 'sndec_in'],
      dtype='object')

In [37]:
host_galaxy_props.rename(columns=dict(snid='snid_in'), inplace=True)

In [41]:
sndf.head()

,htmid_level_6,galaxy_id,c_in,mB,t0_in,x0_in,x1_in,z_in,snid_in,snra_in,sndec_in
0,9021,10562502111,0.035201,18.516489,60772.368515,0.000617,1.477374,0.077278,MS_10199_0,66.115587,-40.866055
1,9023,10562500822,-0.086665,17.966773,63060.297448,0.001028,0.576378,0.073256,MS_10199_1,66.662435,-42.042877
2,9021,10562500735,0.037553,19.078334,62832.166508,0.000370,-0.177457,0.068469,MS_10199_2,65.212146,-41.416473
3,9021,10562502246,0.076715,19.192019,61400.008116,0.000335,-1.344205,0.080189,MS_10199_3,65.469824,-41.109646
4,9023,10562504918,-0.012653,19.388848,61342.657248,0.000278,0.474204,0.115499,MS_10199_5,65.986211,-42.086708


In [40]:
host_galaxy_props.head()

,dec,galaxy_id,morphology/totalSersicIndex,ra,mag_r,snid_in
0,-27.698565,2562500806,3.850073,53.491017,18.892840,MS_9043_2
1,-28.654434,2562500869,1.000000,53.228259,20.105118,MS_9043_3
2,-28.554180,2562504877,3.876497,52.707182,18.045374,MS_9043_7
3,-28.202155,2562504524,2.191405,52.889861,18.638982,MS_9043_8
4,-27.799101,2562504813,1.616776,53.162164,18.168871,MS_9043_9


In [44]:
host_galaxy_props.snid_in.describe()

count             2783
unique            2783
top       MS_9043_3547
freq                 1
Name: snid_in, dtype: object

In [45]:
sndf.snid_in.describe()

count           588212
unique          588212
top       MS_8915_1145
freq                 1
Name: snid_in, dtype: object

In [57]:
xx.to_csv('~/joined_gal_sn_ddf.csv', index=False)

In [60]:
from lsst.sims.utils import angularSeparation

ModuleNotFoundError: No module named 'lsst'